In [2]:
!pip install mediapipe
!pip install cvzone

In [1]:
import cv2
import numpy
import mediapipe as mp
import cvzone
import os
from cvzone.HandTrackingModule import HandDetector
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [2]:
class DragImg():
    def __init__(self, path, posOrigin, imgType):
 
        self.posOrigin = posOrigin
        self.imgType = imgType
        self.path = path
 
        if self.imgType == 'png':
            self.img = cv2.imread(self.path, cv2.IMREAD_UNCHANGED)
        else:
            self.img = cv2.imread(self.path)
 
        self.img = cv2.resize(self.img, (0,0),None,0.2,0.2)
 
        self.size = self.img.shape[:2]
 
    def update(self, cursor):
        ox, oy = self.posOrigin
        h, w = self.size
 
        # Check if in region
        if ox < cursor[0] < ox + w and oy < cursor[1] < oy + h:
            self.posOrigin = cursor[0] - w // 2, cursor[1] - h // 2

In [3]:
path = 'images'
paths = os.listdir(path)

In [4]:
paths

['dog.png', 'flowers.png', 'sandwich.png']

In [5]:
images = []
for x, img_path in enumerate(paths):
    if 'png' in img_path:
        img_type = 'png'
    else:
        img_type = 'jpg'
    images.append(DragImg(f'{path}/{img_path}', [50 + x * 300, 50], img_type))

In [6]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)
 
detector = HandDetector(detectionCon=0.8)

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)
    rect1 = cv2.rectangle(img, (1200,400), (900, 700), (255, 0, 0), 2)
    rect2 = cv2.rectangle(img, (1200,50), (900, 350), (0, 255, 0), 2)
    if hands:
        lmList = hands[0]['lmList']
        print(lmList)
        print(lmList[8][:2])
        print(lmList[12][:2])
        # Check if clicked
        length, info, img = detector.findDistance(lmList[8][:2], lmList[12][:2], img)
        print(length)
        if length < 60:
            cursor = lmList[8]
            for img_obj in images:
                img_obj.update(cursor)

    try:
        for img_obj in images:

            # Draw for JPG image
            h, w = img_obj.size
            ox, oy = img_obj.posOrigin
            if img_obj.imgType == "png":
                # Draw for PNG Images
                img = cvzone.overlayPNG(img, img_obj.img, [ox, oy])
            else:
                img[oy:oy + h, ox:ox + w] = img_obj.img

    except:
        pass

    cv2.imshow("Image", img)
    cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[/Users/build/sandbox/BuildRepos/Sol-Windows-1/External/VirtualMonitor-macOS/MersiveRelayCam/RelayPlugin/VtcComLib/VtcComLib.cpp] initializer()
[/Users/build/sandbox/BuildRepos/Sol-Windows-1/External/VirtualMonitor-macOS/MersiveRelayCam/RelayPlugin/avCam/avCam/avCamLib.mm] initializer()
	05-28-2022  17:38:05.182 @@@@===RelayStream:: set kCMIOStreamPropertyFormatDescription sub=2vuy w=1280 h=720
	05-28-2022  17:38:05.577 LocalCamera::passResolution set resolution to 1280x720
	05-28-2022  17:38:05.621 Local Camera start, ret=0
	05-28-2022  17:38:05.816 @@@@===RelayStream:: set kCMIOStreamPropertyFormatDescription sub=2vuy w=1280 h=720
	05-28-2022  17:38:05.139 Local Camera start, ret=0


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[[383, 662, 0], [477, 630, 22], [544, 573, 28], [600, 545, 22], [649, 528, 16], [484, 463, 51], [569, 461, 33], [621, 489, 11], [660, 518, -3], [464, 450, 22], [574, 448, -4], [630, 486, -28], [671, 526, -40], [449, 452, -9], [562, 448, -38], [626, 485, -54], [670, 524, -58], [441, 468, -40], [536, 450, -62], [594, 465, -70], [639, 487, -71]]
[660, 518]
[671, 526]
13.601470508735444
[[423, 710, 0], [533, 675, 40], [596, 631, 52], [650, 611, 44], [696, 600, 36], [532, 509, 80], [622, 513, 65], [672, 548, 41], [704, 586, 25], [516, 497, 41], [630, 507, 21], [680, 551, -3], [706, 599, -15], [508, 504, 0], [623, 505, -21], [676, 550, -33], [704, 594, -33], [510, 528, -39], [605, 509, -53], [658, 532, -51], [692, 560, -45]]
[704, 586]
[706, 599]
13.152946437965904
[[421, 643, 0], [500, 602, 12], [550, 549, 15], [597, 517, 6], [637, 493, 0], [469, 457, 55], [540, 437, 33], [594, 458, 7], [632, 484, -7], [453, 447, 32], [549, 426, 3], [607, 453, -21], [647, 487, -32], [443, 445, 3], [550, 424

KeyboardInterrupt: 